In [53]:
import torch.nn.functional as F
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification

In [2]:
sent_cls_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

In [3]:
sent_cls = pipeline("sentiment-analysis", model=sent_cls_checkpoint)

In [9]:
sent_cls_raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

In [10]:
res = sent_cls(sent_cls_raw_inputs)

In [11]:
print(res)

[{'label': 'POSITIVE', 'score': 0.9598049521446228}, {'label': 'NEGATIVE', 'score': 0.9994558691978455}]


# Tokenization

In [12]:
tokenizer = AutoTokenizer.from_pretrained(sent_cls_checkpoint)

In [27]:
sent_cls_prep_inputs = tokenizer(sent_cls_raw_inputs, padding=True, truncation=True, return_tensors="pt")

The tokens list usually includes [CLS] at the beginning and [SEP] at the end

In [28]:
sent_cls_prep_inputs

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}

# Model for hidden state

In [41]:
model1 = AutoModel.from_pretrained(sent_cls_checkpoint)

In [42]:
outputs = model1(**sent_cls_prep_inputs)

In [43]:
outputs.last_hidden_state.shape

torch.Size([2, 16, 768])

# Model for classification

In [55]:
model2 = AutoModelForSequenceClassification.from_pretrained(sent_cls_checkpoint)

In [60]:
with torch.no_grad():
    outputs = model2(**sent_cls_prep_inputs)

In [61]:
outputs.logits.shape

torch.Size([2, 2])

In [62]:
predictions = F.softmax(outputs.logits, dim=-1)

In [63]:
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]])


In [64]:
model2.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}